# Sequencing two machines - Modeling
#### José María Benítez, Pablo Berástegui, Daniel Escobosa y Alejandro de Haro

### Importing packages and modules


In [15]:
# module for building the pyomo model
import pyomo.environ as pe
# module for solving the pyomo model
import pyomo.opt as po


### Create the model


In [16]:
model = pe.ConcreteModel()

Order to build the model:
1. Sets
1. Parameters
1. Variables
1. Objective function
1. Constraints


#### Sets

$t$: tasks {T1,T2,T3,T4,T5}  
$m$: machines {M1,M2}


In [17]:
# Tasks and machines (you can edit these lists if needed)
model.tasks = pe.Set(initialize=['T1','T2','T3','T4','T5'])
model.machines = pe.Set(initialize=['M1','M2'])


#### Parameters


$T_{t,m}$: processing time of task $t$ in machine $m$ [h]  
$D_t$: due date (delivery time) of task $t$ [h]


In [18]:
# --- Default instance (replace these dictionaries with your data) ---
# Processing times T_{t,m} in hours
processing_time = {
    ('T1','M1'): 4, ('T1','M2'): 7,
    ('T2','M1'): 3, ('T2','M2'): 5,
    ('T3','M1'): 5, ('T3','M2'): 3,
    ('T4','M1'): 6, ('T4','M2'): 4,
    ('T5','M1'): 2, ('T5','M2'): 6,
}

# Due dates D_t in hours (from time 0)
due_date = {
    'T1': 18,
    'T2': 16,
    'T3': 12,
    'T4': 20,
    'T5': 15,
}

# Params
model.T = pe.Param(model.tasks, model.machines, initialize=processing_time)
model.D = pe.Param(model.tasks, initialize=due_date)


#### Variables


$s_{t,m}$: starting time of task $t$ in machine $m$ [h]  
$x_{t,t',m}$: 1 if task $t$ is done **before** task $t'$ in machine $m$; 0 otherwise  
$p_t$: tardiness (delay) of task $t$ [h]  
$mx$: maximum tardiness over all tasks [h]  
$end$: maximum completion time over all tasks (makespan on M2) [h]


In [19]:
# Start times
model.s = pe.Var(model.tasks, model.machines, within=pe.NonNegativeReals)

# Ordering binaries for all ordered pairs t != t' on each machine
model.pairs = pe.Set(initialize=[(i,j) for i in model.tasks for j in model.tasks if i != j])
model.x = pe.Var(model.pairs, model.machines, within=pe.Binary)

# Tardiness, maximum tardiness, and end time
model.p = pe.Var(model.tasks, within=pe.NonNegativeReals)
model.mx = pe.Var(within=pe.NonNegativeReals)
model.end = pe.Var(within=pe.NonNegativeReals)


#### Objective Function

Minimize maximum delay of all tasks [h]

$\min\; mx$


In [20]:
def obj_rule(model):
    return model.mx

model.obj = pe.Objective(rule=obj_rule, sense=pe.minimize)


#### Constraints


Task $t$ in machine M2 after finishing in machine M1 [h]  
$ s_{t,M2} \ge s_{t,M1} + T_{t,M1} \quad \forall t $


In [21]:
model.flow = pe.ConstraintList()
for t in model.tasks:
    model.flow.add(model.s[t,'M2'] >= model.s[t,'M1'] + model.T[t,'M1'])


Task $t$ and task $t'$ cannot overlap in the same machine $m$ [h]  
If $x_{t,t',m} = 1 \Rightarrow s_{t',m} \ge s_{t,m} + T_{t,m}$ and vice versa; enforce exclusivity  
$ x_{t,t',m} + x_{t',t,m} = 1 \quad \forall t\ne t', m $


In [22]:
# Big-M per machine: sum of all processing times on that machine (safe upper bound for completion time)
M_by_m = {m: sum(pe.value(model.T[t,m]) for t in model.tasks) for m in model.machines}

# Disjunctive ordering and non-overlap
model.n_overlap = pe.ConstraintList()
model.ordering = pe.ConstraintList()

for (t, tp) in model.pairs:
    for m in model.machines:
        M = M_by_m[m]
        # If t before tp on m (x[t,tp,m] = 1), then start(tp,m) >= start(t,m) + T[t,m]
        model.n_overlap.add(model.s[tp, m] >= model.s[t, m] + model.T[t, m] - M * (1 - model.x[(t, tp), m]))
        # Exclusivity with the reverse order
        if t < tp:  # add each pair once (using string order)
            model.ordering.add(model.x[(t, tp), m] + model.x[(tp, t), m] == 1)


Task 3 needs to be processed before task 2 (on every machine)  
Option A: precedence with times  
$ s_{T2,m} \ge s_{T3,m} + T_{T3,m} \quad \forall m $
Option B: fix ordering binaries $x_{T3,T2,m}=1$.  
(Here we use **A** to keep the model simple.)


In [23]:
model.pre_T3_before_T2 = pe.ConstraintList()
for m in model.machines:
    model.pre_T3_before_T2.add(model.s['T2', m] >= model.s['T3', m] + model.T['T3', m])


Delay (tardiness) of each task $t$  
$ p_t \ge s_{t,M2} + T_{t,M2} - D_t \quad \forall t $  
and maximum delay and maximum delivery date:  
$ mx \ge p_t \quad \forall t, \qquad end \ge s_{t,M2} + T_{t,M2} \quad \forall t $


In [24]:
model.tardiness = pe.ConstraintList()
model.max_delay = pe.ConstraintList()
model.end_def = pe.ConstraintList()

for t in model.tasks:
    model.tardiness.add(model.p[t] >= model.s[t,'M2'] + model.T[t,'M2'] - model.D[t])
    model.max_delay.add(model.mx >= model.p[t])
    model.end_def.add(model.end >= model.s[t,'M2'] + model.T[t,'M2'])


Positive variables

$ s_{t,m} \ge 0 $ (already enforced by variable domain)


#### Solver definition and solve statement


In [25]:
solver = po.SolverFactory('gurobi')
results = solver.solve(model, tee=True)


Set parameter Username
Set parameter LicenseID to value 2708942
Academic license - for non-commercial use only - expires 2026-09-15
Read LP format model from file C:\Users\teach\AppData\Local\Temp\tmpaw41u0fz.pyomo.lp
Reading time = 0.00 seconds
x1: 82 rows, 57 columns, 204 nonzeros
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 82 rows, 57 columns and 204 nonzeros
Model fingerprint: 0x856cf65e
Variable types: 17 continuous, 40 integer (40 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 11.0000000
Presolve removed 32 rows and 28 columns
Presolve time: 0.00s
Presolved: 50 rows, 29 columns, 136 nonzeros
Variable types:

In [26]:
print("Objective (mx = maximum tardiness):", round(pe.value(model.obj), 4))
print("Makespan (end):", round(pe.value(model.end), 4))


Objective (mx = maximum tardiness): 7.0
Makespan (end): 27.0


In [27]:
# Pretty print schedule by machine (sorted by start)
for m in model.machines:
    starts = [(t, pe.value(model.s[t,m]), pe.value(model.s[t,m]) + pe.value(model.T[t,m])) for t in model.tasks]
    starts.sort(key=lambda z: z[1])
    print(f"\nMachine {m}:")
    for t, st, ft in starts:
        print(f"  {t}: start={st:.2f}, finish={ft:.2f}")

print("\nTardiness by task:")
for t in model.tasks:
    print(f"  {t}: p_t = {pe.value(model.p[t]):.2f}")



Machine M1:
  T5: start=0.00, finish=2.00
  T3: start=2.00, finish=7.00
  T2: start=7.00, finish=10.00
  T1: start=10.00, finish=14.00
  T4: start=14.00, finish=20.00

Machine M2:
  T5: start=2.00, finish=8.00
  T3: start=8.00, finish=11.00
  T2: start=11.00, finish=16.00
  T1: start=16.00, finish=23.00
  T4: start=23.00, finish=27.00

Tardiness by task:
  T1: p_t = 7.00
  T2: p_t = 7.00
  T3: p_t = 7.00
  T4: p_t = 7.00
  T5: p_t = 7.00
